# Clusters
Este modulo contiene un conjunto de distintos ejemplos de clusterizaciones, nos sirve para mostrar el potencial de nbdev y como generar software de forma consistente. 

In [ ]:
#| default_exp cluster

In [ ]:
#| hide
from nbdev.showdoc import *
import sklearn
import random 
import numpy as np
import shapely


Vamos hacer una clase para que contenga los datos

In [ ]:
#| export
class data_points:
    """Una clase que contiene a los datos"""
    def __init__(self, n, min_x = 0, min_y = 0, max_x= 1, max_y=1, seed= None):
        if seed != None:
            random.seed(seed)
        
        self.Points = np.random.uniform(low= min_x, high= max_x, size=(n, 2));
        self.Points =  [shapely.Point(x[0],x[1]) for x in self.Points]

    
    def get_points(self):
        """Una funcion que para obtener los puntos"""
        return self.Points

            
    

In [ ]:
#|hide
datos_simples = data_points(40)


[<POINT (0.182 0.566)>, <POINT (0.798 0.51)>, <POINT (0.543 0.504)>, <POINT (0.643 0.017)>, <POINT (0.977 0.168)>, <POINT (0.354 0.572)>, <POINT (0.762 0.663)>, <POINT (0.72 0.382)>, <POINT (0.633 0.913)>, <POINT (0.111 0.086)>, <POINT (0.304 0.685)>, <POINT (0.883 0.697)>, <POINT (0.085 0.689)>, <POINT (0.444 0.915)>, <POINT (0.827 0.17)>, <POINT (0.302 0.741)>, <POINT (0.33 0.948)>, <POINT (0.591 0.675)>, <POINT (0.136 0.859)>, <POINT (0.625 0.418)>, <POINT (0.288 0.177)>, <POINT (0.999 0.968)>, <POINT (0.787 0.373)>, <POINT (0.237 0.656)>, <POINT (0.548 0.57)>, <POINT (0.984 0.939)>, <POINT (0.201 0.653)>, <POINT (0.007 0.296)>, <POINT (0.539 0.849)>, <POINT (0.52 0.158)>, <POINT (0.295 0.925)>, <POINT (0.434 0.977)>, <POINT (0.765 0.079)>, <POINT (0.883 0.55)>, <POINT (0.25 0.924)>, <POINT (0.074 0.924)>, <POINT (0.988 0.596)>, <POINT (0.072 0.282)>, <POINT (0.596 0.978)>, <POINT (0.559 0.01)>]


In [ ]:
#| hide
#| output: false
print(datos_simples.get_points())

[<POINT (0.182 0.566)>, <POINT (0.798 0.51)>, <POINT (0.543 0.504)>, <POINT (0.643 0.017)>, <POINT (0.977 0.168)>, <POINT (0.354 0.572)>, <POINT (0.762 0.663)>, <POINT (0.72 0.382)>, <POINT (0.633 0.913)>, <POINT (0.111 0.086)>, <POINT (0.304 0.685)>, <POINT (0.883 0.697)>, <POINT (0.085 0.689)>, <POINT (0.444 0.915)>, <POINT (0.827 0.17)>, <POINT (0.302 0.741)>, <POINT (0.33 0.948)>, <POINT (0.591 0.675)>, <POINT (0.136 0.859)>, <POINT (0.625 0.418)>, <POINT (0.288 0.177)>, <POINT (0.999 0.968)>, <POINT (0.787 0.373)>, <POINT (0.237 0.656)>, <POINT (0.548 0.57)>, <POINT (0.984 0.939)>, <POINT (0.201 0.653)>, <POINT (0.007 0.296)>, <POINT (0.539 0.849)>, <POINT (0.52 0.158)>, <POINT (0.295 0.925)>, <POINT (0.434 0.977)>, <POINT (0.765 0.079)>, <POINT (0.883 0.55)>, <POINT (0.25 0.924)>, <POINT (0.074 0.924)>, <POINT (0.988 0.596)>, <POINT (0.072 0.282)>, <POINT (0.596 0.978)>, <POINT (0.559 0.01)>]


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()